### Module: AURORA.PY

In [ ]:
"""
Main aurora module
=====================
.......................................................................
The functions in this module allow the post-processing of
hydrodynamical simulations to create mock H-alpha observations using
the whole physical and geometrical information of the particles in the
simulation.

Examples::
    ------ to be added ------
"""

In [ ]:
import gc
import re
import os
import sys
import pdb
import math
import time
import logging
import pynbody
import warnings
import numpy as np
import configparser
import scipy.fftpack
from scipy import ndimage
from pympler import asizeof
from astropy.io import fits
import multiprocessing as mp
from scipy.stats import norm
import matplotlib.pyplot as plt
from astropy import units as unit
from sklearn.neighbors import KDTree
from astropy import constants as const
from astropy.cosmology import Planck13 as cosmo

In [ ]:
from . import presets
from . import constants as ct
from . import set_output as so
from . import datacube as dc
from . import snapshot_tools as snap
from . import spectrum_tools as spec
from . import configuration as config
from . import array_operations as arr
from . import gasProps_sBird as bird

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
def __setup_logging():
    logging.basicConfig(
        format="%(levelname)s:%(message)s", 
        filename="aurora.log", 
        level=logging.INFO)

In [ ]:
def __aurora_version():
    """
    Print the version of Aurora being used for future references.
    """
    print("   ___               ")
    print("  / _ |__ _________  _______ _   ")
    print(" / __ / // / __/ _ \/ __/ _ `/   ")
    print("/_/ |_\___/_/  \___/_/  \___/    ")
    print("////// Version 2.1")

In [ ]:
def spectrom_mock(ConfigFile):
    """
    Map the estimated H-alpha flux from the simulation to a mock data
    cube and stores the output in fits format.

    Parameters
    ----------
    ConfigFile : location of the configuration file containing the input
        parameters needed.
    """
    __setup_logging()
    __aurora_version()

    # Code flow:
    # =====================
    # > Load the input parameters from ConfigFile
    # > Read the snapshot
    # > Set geometrical orientation and retain only the desired gas particles
    geom, run, spectrom = config.get_allinput(ConfigFile)
    data = snap.read_snap(run.input_file)
    data_gas = snap.set_snapshots_ready(geom, run, data)[0]
    del data
    gc.collect()
    
    # > Retain only those gas particles wich lie inside the field of view
    lim = spectrom.fieldofview.to("kpc").value/2.
    data_gas = snap.filter_array(data_gas,["x","y"],2*[-lim],2*[lim],2*["kpc"])
    
        # Code flow:
    # =====================
    # > Determine the smoothing lengths
    # > Increase target resolution to minimize geometrical concerns
    # > Compute the fluxes separately for each AMR scale
    # > Smooth the fluxes from each scale and collapse them
    snap.set_hsml_limits(run, data_gas)
    spectrom.oversample()
    cube = spec.__project_all_chunks(geom, run, spectrom, data_gas)
    spec.__cube_convolution(geom, run, spectrom, cube)
    cube = np.sum(cube, axis=3)

    # Code flow:
    # =====================
    # > Bin to recover the target spatial sampling
    # > Inject noise
    # > Store the final datacube

    cube = arr.bin_array(cube, spectrom.oversampling, axis=1)
    cube = arr.bin_array(cube, spectrom.oversampling, axis=2)
    spectrom.undersample()
    
    if(spectrom.sigma_cont > 0.):
        logging.info(f"// Noise injection")
        cube_noise = cube + np.random.normal(0.0, spectrom.sigma_cont, cube.shape)

    logging.info(f"Created file {run.output_name}")
    so.writing_datacube(geom, spectrom, run, cube)